# Results of all benchmark (yolov8s)

результаты приведены для val части `Russian traffic sign images dataset (RSTD)` c `Kaggle` ([ссылка](https://www.kaggle.com/datasets/watchman/rtsd-dataset)) для детекции дорожных знаков России с сохранением 22 самых популярных классов дорожных знаков.

| Device-CPU          | preprocess ms | inference ms | loss ms | postprocess ms | mask map50-95 | mask map50 | mask map75 | size Mb|
|---------------------|---------------|--------------|---------|----------------|---------------|------------|------------|--------|
| PyTorch fuse        | 0.1           | 37.0         | 0       | 0.3            | 0.742         | 0.97       | 0.893      | 21.5   |
| ONNX                | 0.4           | 62.0         | 0       | 1.8            | 0.738         | 0.967      | 0.888      | 42.7   |
| ONNX Simplify       | 0.3           | 66.8         | 0       | 0.8            | 0.738         | 0.967      | 0.888      | 42.6   |
| ONNX Simplify half  | 0.4           | 62.7         | 0       | 0.9            | 0.738         | 0.967      | 0.888      | 42.6   |
| OpenVino            | 0.3           | 45.1         | 0       | 0.4            | 0.738         | 0.967      | 0.888      | 42.8   |
| OpenVino half       | 0.3           | 44.7         | 0       | 0.4            | 0.738         | 0.967      | 0.888      | 21.6   |


# PyTorch Fuse

In [1]:
import ultralytics
import shutil

ultralytics.checks()

Ultralytics YOLOv8.0.212 🚀 Python-3.10.11 torch-1.13.0+cu117 CUDA:0 (NVIDIA A10, 24043MiB)
Setup complete ✅ (12 CPUs, 31.1 GB RAM, 436.5/913.8 GB disk)


In [2]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 151
model name	: 12th Gen Intel(R) Core(TM) i5-12600
stepping	: 5
microcode	: 0x2e
cpu MHz		: 3300.000
cache size	: 18432 KB
physical id	: 0
siblings	: 12
core id		: 0
cpu cores	: 6
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 32
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc art arch_perfmon pebs bts rep_good nopl xtopology nonstop_tsc cpuid aperfmperf tsc_known_freq pni pclmulqdq dtes64 monitor ds_cpl vmx smx est tm2 ssse3 sdbg fma cx16 xtpr pdcm sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand lahf_lm abm 3dnowprefetch cpuid_fault epb ssbd ibrs ibpb stibp ibrs_enhanced tpr_shadow vnmi flexpriority ept vpid ept_ad fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid rdseed adx smap clflushopt clwb intel_pt sha_ni xsaveopt xsavec x

In [3]:
import os

print(os.getcwd())

/home/admin-gpu/Downloads/drivers_helper/notebooks


In [4]:
model_path = "../models/models/yolov8s_640/weights/best.pt"
dataset_yaml_path = "../data/processed/RSTD_filtered_yolov5/dataset.yaml"

In [5]:
from ultralytics import YOLO
import os

# load a pretrained model (recommended for training)
model = YOLO(model_path)

## Измерим параметры на ЦПУ

In [6]:
# Validate the model
# При валлидации происходит Fusing слоев модели!
# после layer fusion, стало 168 слоев !
# уменьшение слоев примерно на 25%
metrics = model.val(data=dataset_yaml_path, batch=1, imgsz=640, device="cpu")

Ultralytics YOLOv8.0.212 🚀 Python-3.10.11 torch-1.13.0+cu117 CPU (12th Gen Intel Core(TM) i5-12600)
Model summary (fused): 168 layers, 11134098 parameters, 0 gradients, 28.5 GFLOPs


val: Scanning /home/admin-gpu/Downloads/drivers_helper/data/processed/RSTD_filtered_yolov5/labels/val.cache... 4185 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4185/4185 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4185/4185 [03:05<00:00, 22.59it/s]


                   all       4185       6821       0.93       0.93       0.97      0.742
                5_19_1       4185       2117      0.906      0.933      0.971      0.732
                   2_1       4185        934      0.966      0.958      0.988      0.792
                  5_16       4185        445      0.934      0.933      0.973      0.717
                5_15_2       4185        440      0.941      0.973      0.987      0.759
                  3_24       4185        343      0.928      0.913      0.959      0.732
                   2_4       4185        340      0.938      0.953       0.98      0.735
                  3_27       4185        290       0.93      0.959      0.976      0.756
                  1_23       4185        229      0.937      0.913      0.978      0.729
                 4_1_1       4185        196       0.92      0.908      0.963      0.735
                  5_20       4185        154      0.946      0.916      0.965      0.739
                  3_2

In [13]:
# оценим производительность
print(f'preprocess {metrics.speed["preprocess"]:.2} ms')
print(f'inference {metrics.speed["inference"]:.2} ms')
print(f'loss {metrics.speed["loss"]:.2} ms')
print(f'postprocess {metrics.speed["postprocess"]:.2} ms')

# оценим метрики для сегментации
print(f"mask map50-95 {metrics.box.map:.3}")
print(f"mask map50 {metrics.box.map50:.3}")
print(f"mask map75 {metrics.box.map75:.3}")

preprocess 0.14 ms
inference 3.6e+01 ms
loss 0.0044 ms
postprocess 0.28 ms
mask map50-95 0.742
mask map50 0.97
mask map75 0.893


In [9]:
# оценим размер сериализованной модели
model_size = os.path.getsize(model_path) / 1024**2
print(f"{model_size:.3} Мб")

21.5 Мб


# Экспорт моделей в различные форматы

Согласно официальной документации и туториалу `YOLOv8` поддерживает экспорт в множество различных форматов. 

Export a YOLOv8 model to any supported format below with the `format` argument, i.e. `format=onnx`. See [YOLOv8 Export Docs](https://docs.ultralytics.com/modes/export/) for more information.

- 💡 ProTip: Export to [ONNX](https://onnx.ai/) or [OpenVINO](https://docs.openvino.ai/latest/index.html) for up to 3x CPU speedup.  
- 💡 ProTip: Export to [TensorRT](https://developer.nvidia.com/tensorrt) for up to 5x GPU speedup.


| Format                                                             | `format` Argument | Model                     | Metadata | Arguments                                           |
|--------------------------------------------------------------------|-------------------|---------------------------|----------|-----------------------------------------------------|
| [PyTorch](https://pytorch.org/)                                    | -                 | `yolov8n.pt`              | ✅        | -                                                   |
| [TorchScript](https://pytorch.org/docs/stable/jit.html)            | `torchscript`     | `yolov8n.torchscript`     | ✅        | `imgsz`, `optimize`                                 |
| [ONNX](https://onnx.ai/)                                           | `onnx`            | `yolov8n.onnx`            | ✅        | `imgsz`, `half`, `dynamic`, `simplify`, `opset`     |
| [OpenVINO](https://docs.openvino.ai/latest/index.html)             | `openvino`        | `yolov8n_openvino_model/` | ✅        | `imgsz`, `half`                                     |
| [TensorRT](https://developer.nvidia.com/tensorrt)                  | `engine`          | `yolov8n.engine`          | ✅        | `imgsz`, `half`, `dynamic`, `simplify`, `workspace` |
| [CoreML](https://github.com/apple/coremltools)                     | `coreml`          | `yolov8n.mlpackage`       | ✅        | `imgsz`, `half`, `int8`, `nms`                      |
| [TF SavedModel](https://www.tensorflow.org/guide/saved_model)      | `saved_model`     | `yolov8n_saved_model/`    | ✅        | `imgsz`, `keras`                                    |
| [TF GraphDef](https://www.tensorflow.org/api_docs/python/tf/Graph) | `pb`              | `yolov8n.pb`              | ❌        | `imgsz`                                             |
| [TF Lite](https://www.tensorflow.org/lite)                         | `tflite`          | `yolov8n.tflite`          | ✅        | `imgsz`, `half`, `int8`                             |
| [TF Edge TPU](https://coral.ai/docs/edgetpu/models-intro/)         | `edgetpu`         | `yolov8n_edgetpu.tflite`  | ✅        | `imgsz`                                             |
| [TF.js](https://www.tensorflow.org/js)                             | `tfjs`            | `yolov8n_web_model/`      | ✅        | `imgsz`                                             |
| [PaddlePaddle](https://github.com/PaddlePaddle)                    | `paddle`          | `yolov8n_paddle_model/`   | ✅        | `imgsz`                                             |
| [ncnn](https://github.com/Tencent/ncnn)                            | `ncnn`            | `yolov8n_ncnn_model/`     | ✅        | `imgsz`, `half`                                     |


# Экспорт в ONNX

## ONNX only

In [14]:
# создадим чистую исходную модель

from ultralytics import YOLO
import os

# load a pretrained model (recommended for training)
model = YOLO(model_path)

In [18]:
model.export(format="onnx")

ValueError: Invalid export format='-'. Valid formats are ('torchscript', 'onnx', 'openvino', 'engine', 'coreml', 'saved_model', 'pb', 'tflite', 'edgetpu', 'tfjs', 'paddle', 'ncnn')

In [13]:
# Load a model
model_onnx_path = "../models/train/weights/best.onnx"
model = YOLO(model_onnx_path)

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify', or 'pose'.


## Измерим параметры на ЦПУ

In [14]:
metrics = model.val(data=dataset_yaml_path, batch=1, imgsz=640, device="cpu")

Ultralytics YOLOv8.0.212 🚀 Python-3.10.11 torch-1.13.0+cu117 CPU (12th Gen Intel Core(TM) i5-12600)
Loading ../models/train/weights/best.onnx for ONNX Runtime inference...
Forcing batch=1 square inference (1,3,640,640) for non-PyTorch models


val: Scanning /home/admin-gpu/Downloads/drivers_helper/data/processed/RSTD_filtered_yolov5/labels/val.cache... 4185 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4185/4185 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4185/4185 [05:02<00:00, 13.85it/s]


                   all       4185       6821      0.935      0.924      0.967      0.738
                5_19_1       4185       2117        0.9      0.932      0.965      0.727
                   2_1       4185        934      0.966      0.961      0.988      0.789
                  5_16       4185        445       0.93      0.931      0.971      0.713
                5_15_2       4185        440      0.944      0.977      0.987      0.756
                  3_24       4185        343      0.931      0.913      0.953      0.726
                   2_4       4185        340      0.939      0.953      0.982      0.736
                  3_27       4185        290      0.942      0.954      0.977      0.758
                  1_23       4185        229      0.953      0.913      0.979       0.73
                 4_1_1       4185        196       0.93      0.908      0.963      0.738
                  5_20       4185        154       0.95       0.89      0.967      0.737
                  3_2

In [17]:
# оценим метрики для сегментации
print(f"mask map50-95 {metrics.box.map:.3}")
print(f"mask map50 {metrics.box.map50:.3}")
print(f"mask map75 {metrics.box.map75:.3}")

mask map50-95 0.738
mask map50 0.967
mask map75 0.888


In [18]:
# оценим размер сериализованной модели
model_size = os.path.getsize(model_onnx_path) / 1024**2
print(f"{model_size:.3} Мб")

42.7 Мб


In [19]:
# остальные методы сохраняют модель под таким же названием
os.remove(model_onnx_path)

## ONNX simplify

In [5]:
# создадим чистую исходную модель

from ultralytics import YOLO
import os

# load a pretrained model (recommended for training)
model = YOLO(model_path)

In [6]:
model.export(format="onnx", simplify=True)

Ultralytics YOLOv8.0.212 🚀 Python-3.10.11 torch-1.13.0+cu117 CPU (12th Gen Intel Core(TM) i5-12600)
Model summary (fused): 168 layers, 11134098 parameters, 0 gradients, 28.5 GFLOPs

PyTorch: starting from '../models/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 26, 8400) (21.5 MB)

ONNX: starting export with onnx 1.15.0 opset 16...
ONNX: simplifying with onnxsim 0.4.35...
ONNX: export success ✅ 0.9s, saved as '../models/train/weights/best.onnx' (42.6 MB)

Export complete (2.5s)
Results saved to /home/admin-gpu/Downloads/drivers_helper/models/train/weights
Predict:         yolo predict task=detect model=../models/train/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=../models/train/weights/best.onnx imgsz=640 data=data/RSTD_filtered_yolov5/dataset.yaml  
Visualize:       https://netron.app


'../models/train/weights/best.onnx'

In [7]:
# Load a model
model_onnx_path = "../models/train/weights/best.onnx"
model = YOLO(model_onnx_path)

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify', or 'pose'.


### Измерим параметры на ЦПУ

In [8]:
metrics = model.val(data=dataset_yaml_path, batch=1, imgsz=640, device="cpu")

Ultralytics YOLOv8.0.212 🚀 Python-3.10.11 torch-1.13.0+cu117 CPU (12th Gen Intel Core(TM) i5-12600)
Loading ../models/train/weights/best.onnx for ONNX Runtime inference...
Forcing batch=1 square inference (1,3,640,640) for non-PyTorch models


val: Scanning /home/admin-gpu/Downloads/drivers_helper/data/processed/RSTD_filtered_yolov5/labels/val.cache... 4185 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4185/4185 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4185/4185 [05:18<00:00, 13.15it/s]


                   all       4185       6821      0.935      0.924      0.967      0.738
                5_19_1       4185       2117        0.9      0.932      0.965      0.727
                   2_1       4185        934      0.966      0.961      0.988      0.789
                  5_16       4185        445       0.93      0.931      0.971      0.713
                5_15_2       4185        440      0.944      0.977      0.987      0.756
                  3_24       4185        343      0.931      0.913      0.953      0.726
                   2_4       4185        340      0.939      0.953      0.982      0.736
                  3_27       4185        290      0.942      0.954      0.977      0.758
                  1_23       4185        229      0.953      0.913      0.979       0.73
                 4_1_1       4185        196       0.93      0.908      0.963      0.738
                  5_20       4185        154       0.95       0.89      0.967      0.737
                  3_2

In [9]:
# оценим метрики для сегментации
print(f"mask map50-95 {metrics.box.map:.3}")
print(f"mask map50 {metrics.box.map50:.3}")
print(f"mask map75 {metrics.box.map75:.3}")

mask map50-95 0.738
mask map50 0.967
mask map75 0.888


In [10]:
# оценим размер сериализованной модели
model_size = os.path.getsize(model_onnx_path) / 1024**2
print(f"{model_size:.3} Мб")

42.6 Мб


In [11]:
# остальные методы сохраняют модель под таким же названием
os.remove(model_onnx_path)

## ONNX simplify half

In [12]:
# создадим чистую исходную модель

from ultralytics import YOLO
import os

# load a pretrained model (recommended for training)
model = YOLO(model_path)

In [13]:
model.export(format="onnx", simplify=True, half=True)

Ultralytics YOLOv8.0.212 🚀 Python-3.10.11 torch-1.13.0+cu117 CPU (12th Gen Intel Core(TM) i5-12600)
WARNING ⚠️ half=True only compatible with GPU export, i.e. use device=0
Model summary (fused): 168 layers, 11134098 parameters, 0 gradients, 28.5 GFLOPs

PyTorch: starting from '../models/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 26, 8400) (21.5 MB)

ONNX: starting export with onnx 1.15.0 opset 16...
ONNX: simplifying with onnxsim 0.4.35...
ONNX: export success ✅ 0.9s, saved as '../models/train/weights/best.onnx' (42.6 MB)

Export complete (2.2s)
Results saved to /home/admin-gpu/Downloads/drivers_helper/models/train/weights
Predict:         yolo predict task=detect model=../models/train/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=../models/train/weights/best.onnx imgsz=640 data=data/RSTD_filtered_yolov5/dataset.yaml  
Visualize:       https://netron.app


'../models/train/weights/best.onnx'

In [14]:
# Load a model
model_onnx_path = "../models/train/weights/best.onnx"
model = YOLO(model_onnx_path)

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify', or 'pose'.


### Измерим параметры на ЦПУ

In [15]:
metrics = model.val(data=dataset_yaml_path, batch=1, imgsz=640, device="cpu")

Ultralytics YOLOv8.0.212 🚀 Python-3.10.11 torch-1.13.0+cu117 CPU (12th Gen Intel Core(TM) i5-12600)
Loading ../models/train/weights/best.onnx for ONNX Runtime inference...
Forcing batch=1 square inference (1,3,640,640) for non-PyTorch models


val: Scanning /home/admin-gpu/Downloads/drivers_helper/data/processed/RSTD_filtered_yolov5/labels/val.cache... 4185 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4185/4185 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4185/4185 [05:03<00:00, 13.77it/s]


                   all       4185       6821      0.935      0.924      0.967      0.738
                5_19_1       4185       2117        0.9      0.932      0.965      0.727
                   2_1       4185        934      0.966      0.961      0.988      0.789
                  5_16       4185        445       0.93      0.931      0.971      0.713
                5_15_2       4185        440      0.944      0.977      0.987      0.756
                  3_24       4185        343      0.931      0.913      0.953      0.726
                   2_4       4185        340      0.939      0.953      0.982      0.736
                  3_27       4185        290      0.942      0.954      0.977      0.758
                  1_23       4185        229      0.953      0.913      0.979       0.73
                 4_1_1       4185        196       0.93      0.908      0.963      0.738
                  5_20       4185        154       0.95       0.89      0.967      0.737
                  3_2

In [16]:
# оценим метрики для сегментации
print(f"mask map50-95 {metrics.box.map:.3}")
print(f"mask map50 {metrics.box.map50:.3}")
print(f"mask map75 {metrics.box.map75:.3}")

mask map50-95 0.738
mask map50 0.967
mask map75 0.888


In [17]:
# оценим размер сериализованной модели
model_size = os.path.getsize(model_onnx_path) / 1024**2
print(f"{model_size:.3} Мб")

42.6 Мб


In [18]:
# остальные методы сохраняют модель под таким же названием
os.remove(model_onnx_path)

# Экспорт в OpenVino

## OpenVino

In [5]:
# создадим чистую исходную модель

from ultralytics import YOLO
import os

# load a pretrained model (recommended for training)
model = YOLO(model_path)

In [6]:
model.export(format="openvino")

Ultralytics YOLOv8.0.212 🚀 Python-3.10.11 torch-1.13.0+cu117 CPU (12th Gen Intel Core(TM) i5-12600)
Model summary (fused): 168 layers, 11134098 parameters, 0 gradients, 28.5 GFLOPs

PyTorch: starting from '../models/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 26, 8400) (21.5 MB)

ONNX: starting export with onnx 1.15.0 opset 16...
ONNX: export success ✅ 0.5s, saved as '../models/train/weights/best.onnx' (42.7 MB)

OpenVINO: starting export with openvino 2023.2.0-13089-cfd42bd2cb0-HEAD...
OpenVINO: export success ✅ 0.7s, saved as '../models/train/weights/best_openvino_model/' (42.8 MB)

Export complete (2.6s)
Results saved to /home/admin-gpu/Downloads/drivers_helper/models/train/weights
Predict:         yolo predict task=detect model=../models/train/weights/best_openvino_model imgsz=640  
Validate:        yolo val task=detect model=../models/train/weights/best_openvino_model imgsz=640 data=data/RSTD_filtered_yolov5/dataset.yaml  
Visualize:      

'../models/train/weights/best_openvino_model'

In [7]:
# Load a model
model_openvino_path = '../models/train/weights/best_openvino_model'
model = YOLO(model_openvino_path)

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify', or 'pose'.


### Измерим параметры на ЦПУ

In [8]:
metrics = model.val(data=dataset_yaml_path, batch=1, imgsz=640, device="cpu")

Ultralytics YOLOv8.0.212 🚀 Python-3.10.11 torch-1.13.0+cu117 CPU (12th Gen Intel Core(TM) i5-12600)
Loading ../models/train/weights/best_openvino_model for OpenVINO inference...
Forcing batch=1 square inference (1,3,640,640) for non-PyTorch models


val: Scanning /home/admin-gpu/Downloads/drivers_helper/data/processed/RSTD_filtered_yolov5/labels/val.cache... 4185 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4185/4185 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4185/4185 [03:44<00:00, 18.62it/s]


                   all       4185       6821      0.935      0.924      0.967      0.738
                5_19_1       4185       2117        0.9      0.932      0.965      0.727
                   2_1       4185        934      0.966      0.961      0.988      0.789
                  5_16       4185        445       0.93      0.931      0.971      0.713
                5_15_2       4185        440      0.944      0.977      0.987      0.756
                  3_24       4185        343      0.931      0.913      0.953      0.726
                   2_4       4185        340      0.939      0.953      0.982      0.736
                  3_27       4185        290      0.942      0.954      0.977      0.758
                  1_23       4185        229      0.953      0.913      0.979       0.73
                 4_1_1       4185        196       0.93      0.908      0.963      0.738
                  5_20       4185        154       0.95       0.89      0.967      0.737
                  3_2

In [9]:
# оценим метрики для сегментации
print(f"mask map50-95 {metrics.box.map:.3}")
print(f"mask map50 {metrics.box.map50:.3}")
print(f"mask map75 {metrics.box.map75:.3}")

mask map50-95 0.738
mask map50 0.967
mask map75 0.888


In [15]:
def get_files_size_in_mb(directory):
    file_sizes = 0
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            size_in_bytes = os.path.getsize(file_path)
            size_in_mb = size_in_bytes / 1024 / 1024
            file_sizes += size_in_mb
            
    return file_sizes

In [16]:
# оценим размер сериализованной модели
model_size = get_files_size_in_mb(model_openvino_path)
print(f"{model_size:.3} Мб")

42.8 Мб


In [17]:
shutil.rmtree(model_openvino_path)

## OpenVino half

In [18]:
# создадим чистую исходную модель

from ultralytics import YOLO
import os

# load a pretrained model (recommended for training)
model = YOLO(model_path)

In [19]:
model.export(format="openvino", half=True)

Ultralytics YOLOv8.0.212 🚀 Python-3.10.11 torch-1.13.0+cu117 CPU (12th Gen Intel Core(TM) i5-12600)
Model summary (fused): 168 layers, 11134098 parameters, 0 gradients, 28.5 GFLOPs

PyTorch: starting from '../models/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 26, 8400) (21.5 MB)

ONNX: starting export with onnx 1.15.0 opset 16...
ONNX: export success ✅ 0.5s, saved as '../models/train/weights/best.onnx' (42.7 MB)

OpenVINO: starting export with openvino 2023.2.0-13089-cfd42bd2cb0-HEAD...
OpenVINO: export success ✅ 0.1s, saved as '../models/train/weights/best_openvino_model/' (21.6 MB)

Export complete (1.9s)
Results saved to /home/admin-gpu/Downloads/drivers_helper/models/train/weights
Predict:         yolo predict task=detect model=../models/train/weights/best_openvino_model imgsz=640 half 
Validate:        yolo val task=detect model=../models/train/weights/best_openvino_model imgsz=640 data=data/RSTD_filtered_yolov5/dataset.yaml half 
Visualiz

'../models/train/weights/best_openvino_model'

In [20]:
# Load a model
model_openvino_path = '../models/train/weights/best_openvino_model'
model = YOLO(model_openvino_path)

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify', or 'pose'.


### Измерим параметры на ЦПУ

In [21]:
metrics = model.val(data=dataset_yaml_path, batch=1, imgsz=640, device="cpu")

Ultralytics YOLOv8.0.212 🚀 Python-3.10.11 torch-1.13.0+cu117 CPU (12th Gen Intel Core(TM) i5-12600)
Loading ../models/train/weights/best_openvino_model for OpenVINO inference...
Forcing batch=1 square inference (1,3,640,640) for non-PyTorch models


val: Scanning /home/admin-gpu/Downloads/drivers_helper/data/processed/RSTD_filtered_yolov5/labels/val.cache... 4185 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4185/4185 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4185/4185 [03:43<00:00, 18.76it/s]


                   all       4185       6821      0.935      0.923      0.967      0.738
                5_19_1       4185       2117        0.9      0.931      0.965      0.727
                   2_1       4185        934      0.966      0.961      0.988      0.789
                  5_16       4185        445       0.93      0.931      0.971      0.713
                5_15_2       4185        440      0.944      0.977      0.987      0.755
                  3_24       4185        343      0.931      0.913      0.953      0.726
                   2_4       4185        340      0.939      0.953      0.982      0.735
                  3_27       4185        290      0.942      0.954      0.977      0.758
                  1_23       4185        229      0.953      0.913      0.979       0.73
                 4_1_1       4185        196       0.93      0.908      0.963      0.738
                  5_20       4185        154       0.95       0.89      0.967      0.737
                  3_2

In [22]:
# оценим метрики для сегментации
print(f"mask map50-95 {metrics.box.map:.3}")
print(f"mask map50 {metrics.box.map50:.3}")
print(f"mask map75 {metrics.box.map75:.3}")

mask map50-95 0.738
mask map50 0.967
mask map75 0.888


In [23]:
def get_files_size_in_mb(directory):
    file_sizes = 0
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path):
            size_in_bytes = os.path.getsize(file_path)
            size_in_mb = size_in_bytes / 1024 / 1024
            file_sizes += size_in_mb
            
    return file_sizes

In [24]:
# оценим размер сериализованной модели
model_size = get_files_size_in_mb(model_openvino_path)
print(f"{model_size:.3} Мб")

21.6 Мб


In [49]:
shutil.rmtree(model_openvino_path)